In [1]:
from collections import defaultdict, Counter, OrderedDict
import ujson
import pathlib
from pathlib import Path
import sqlite3
import sys
import textwrap
import torch
from tqdm import tqdm
from typing import Dict, List, Set, Union

import unicodedata

import uuid

import numpy as np

%load_ext autoreload
%autoreload 2

from aic_nlp_utils.batch import batch_apply
from aic_nlp_utils.encoding import nfc
from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl
from aic_nlp_utils.fever import fever_detokenize, import_fever_corpus_from_sqlite

from zshot_fact_verify.qa2d.qa2d import SameDocumentNERReplacementGenerator
from zshot_fact_verify.wiki.load import load_corpus, create_corpus_splits, select_nei_context_for_splits, load_nei_ners
from zshot_fact_verify.models.load import load_tokenizer_and_model
from zshot_fact_verify.models.arguments import ModelArguments

In [2]:
SEED = 1234
NER_ROOT = '/mnt/data/factcheck/wiki/cs/20230220/qacg/ner/PAV-ner-CNEC'
WIKI_CORPUS = '/mnt/data/factcheck/wiki/cs/20230220/paragraphs/cswiki-20230220-paragraphs.jsonl'
SPLITS  = [
            {"name": "train", "file": Path(NER_ROOT, "train_ners.json"), "size": 10000},
            {"name": "dev", "file": Path(NER_ROOT, "dev_ners.json"), "size": 1000},
            {"name": "test", "file": Path(NER_ROOT, "test_ners.json"), "size": 1000},
        ]
corpus, corpus_id2idx, corpus_pages = load_corpus(WIKI_CORPUS)
corpus_recs_lst = create_corpus_splits(corpus, corpus_id2idx, SPLITS, SEED)
corpus_recs_lst = select_nei_context_for_splits(corpus, corpus_id2idx, corpus_recs_lst, SEED)

imported 514568 corpus pages.


In [3]:
type(corpus_recs_lst[0][0])

dict

In [4]:
len(corpus_recs_lst)

3

In [5]:
corpus_recs_lst[0][1]

{'id': 'Smil_Flaška_z_Pardubic_5',
 'did': 'Smil_Flaška_z_Pardubic',
 'bid': 5,
 'text': 'Potomci.\nSmil zplodil tři potomky, z nichž byl jeden syn a dvě dcery:\nDílo.\nSmilovi bylo rovněž připisováno autorství několika dalších děl, jež vykazují společné rysy. Jedná se o "Sbírku přísloví", "Rady otce synovi", satiry "Svár vody s vínem" a "Podkoní a žák", "O ženě svárlivé" a "Roudnické umučení". Jan Gebauer či Julius Feifalik Smilovo autorství u těchto děl zpochybňovali, naopak Prokop Miroslav Haškovec Smilovo autorství uvedených básní podporoval. Literární historik Josef Hrabák autorství skladeb neurčoval, namísto toho je zařadil do tzv. Smilovy školy, neboť si jsou vzájemně podobné a příbuzné se Smilovou "Novou radou".',
 'url': 'https://cs.wikipedia.org/wiki?curid=6099',
 'revid': '390445',
 'nei_id': 'Smil_Flaška_z_Pardubic_3',
 'nei_bid': 3,
 'nei_text': 'V roce 1394 se proti králi Václavovi IV. postavila panská jednota, vůdčí roli v tomto uskupení zaujali moravský markrabě Jošt a 

In [6]:
NER_FILE = '/mnt/data/factcheck/wiki/cs/20230220/qacg/ner/PAV-ner-CNEC/dev_ners.json'
NEI_NER_FILE = '/mnt/data/factcheck/wiki/cs/20230220/qacg/ner/PAV-ner-CNEC/nei_dev_ners.json'

original_ners = read_json(NER_FILE)
nei_ners = load_nei_ners(corpus_recs_lst[1], original_ners, NEI_NER_FILE)

In [7]:
from zshot_fact_verify.qg.question_generation import BatchQuestionGenerator, generate_questions

LANG = "cs_CZ"
MODEL_NAME = f"/home/drchajan/devel/python/FC/Zero-shot-Fact-Verification/experiments/qg/google/mt5-large_{LANG}/checkpoint-59000"

model_args = ModelArguments(model_name_or_path=MODEL_NAME)
tokenizer, model, data_collator = load_tokenizer_and_model(model_args, lang=LANG, fp16=True)

batch_question_generator = BatchQuestionGenerator(tokenizer, model, highlight=False, padding=True, device="cpu", debug=False)

In [8]:
%pwd

'/home/drchajan/devel/python/FC/Zero-shot-Fact-Verification'

In [10]:
corpus_recs_lst[1][:10]

[{'id': 'Hertzsprungův–Russellův_diagram_4',
  'did': 'Hertzsprungův–Russellův_diagram',
  'bid': 4,
  'text': 'V jiné verzi diagramu se vynáší efektivní povrchová teplota hvězdy na vodorovné ose a svítivost hvězdy na svislé ose, obvykle v log-log souřadnicích. Tato varianta se používá pro zobrazení teoretických výpočtů vývoje hvězd, mohla by se označovat diagram teplota-svítivost, ale většinou se označuje jako teoretický Hertzsprungův–Russellův diagram. Zvláštností této verze je, že teplota na vodorovné ose se vynáší obráceně - zleva doprava teplota klesá - usnadňuje to porovnání s předchozí verzí.',
  'url': 'https://cs.wikipedia.org/wiki?curid=603640',
  'revid': '46874',
  'nei_id': 'Hertzsprungův–Russellův_diagram_22',
  'nei_bid': 22,
  'nei_text': 'Hvězda o hmotnosti 50 "MS" setrvá na hlavní posloupnosti přibližně 100 milionů let.\nKdyž hvězda spálí veškeré své zásoby jaderného paliva (vodík), začne se vlivem vlastní gravitace hroutit, což zapříčiní další zvýšení teploty. Zahřív

In [13]:
qgs = generate_questions(corpus_recs_lst[1][:2], nei_ners, None, batch_question_generator, nei=True)

100%|██████████| 2/2 [01:35<00:00, 47.99s/it]


In [14]:
qgs

OrderedDict([('Nevěsta_duchů_2',
              {'Altenburg:::P': ['Kdo byl přepaden a zabit?', 'Altenburg'],
               'Landhorstu:::G': ['Kde se nachází hrabě, který má dceru?',
                'Landhorstu'],
               'Altenburga:::P': ['Kdo byl hrabě z Landhorstu, který si zaslíbil nevěstu?',
                'Altenburga'],
               'Starkenberg:::P': ['Kdo je přítelem zavražděného hraběte Altenburga?',
                'Starkenberg'],
               'Luitgarde:::P': ['Jak se jmenuje hrabě z Landhorstu?',
                'Luitgarde'],
               'Luitgardina:::P': ['Kdo je hrabě z Landhorstu a jeho dcera?',
                'Luitgardina'],
               'Langhorst:::P': ['Kdo je hrabě, který je ženatý?',
                'Langhorst']})])

In [ ]:
class ClaimGenerator:
    def __init__(self, replacement_generator, corpus_recs, ner_json, qas_json, QA2D_model_path, lang, device="cuda"):
        # QA2D model object
        print('Loading QA2D module >>>>>>>>')
        model_args = ModelArguments(model_name_or_path=QA2D_model_path)
        self.tokenizer, self.model, data_collator = load_tokenizer_and_model(model_args, lang=lang)
        print(f'Running on device: {device}')
        # self.model, self.tokenizer = model, tokenizer # TODO REMOVE
        self.device = device
        self.model.to(device)

        self.replacement_generator = replacement_generator

        self.corpus_recs = corpus_recs
        self.ners = read_json(ner_json)
        self.qas = read_json(qas_json)

    def predict(self, inputs, max_source_length=1024, batch_size=16):
        def pred_func(input_texts: List[str]) -> List[str]:
            with torch.no_grad():
                X = self.tokenizer(input_texts, max_length=max_source_length, padding=True, truncation=True, return_tensors="pt")
                X = {k: X[k].to(self.device) for k in X.keys()}
                Y = self.model.generate(**X, max_new_tokens=768)
                output_texts = self.tokenizer.batch_decode(
                    Y, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
            return output_texts
            
        predictions = batch_apply(pred_func, inputs, batch_size=batch_size)
        return predictions

    def _load_passage_entities(self, id_):
        passage_entities = []
        for ent_text, ent_type, _ in self.ners[id_]:
            passage_entities.append(f'{ent_text}:::{ent_type}') # group by entity name and type as in the QAS file
        return passage_entities
    
    def _load_precomputed_qas_for_entities(self, id_, passage_entities):
        if id_ not in self.qas:
            print(f"missing id: {id_}")
            return None
        QA_for_sample = self.qas[id_]
        QA_pairs = []
        for entity in passage_entities:
            if entity in QA_for_sample:
                ent_text, ent_type = entity.split(':::')
                question, answer = QA_for_sample[entity]
                QA_pairs.append({'question': question, 'answer': answer, 'answer_type': ent_type})
            else:
                print(f"missing entity: {entity} for id: {id_}")
                return None
        if len(QA_pairs) == 0:
            print(f"zero length pairs for id: {id_}")
            return None
        return QA_pairs 
        

    def generate_supported_claims(self, sample):
        texts, id_ = sample['text'], str(sample['id'])

        # Step 1: load entities in text
        passage_entities = self._load_passage_entities(id_)
        if len(passage_entities) == 0: # no NERs
            return None 

        # Step 2: load precomputed QAs for entities
        QA_pairs = self._load_precomputed_qas_for_entities(id_, passage_entities)
        if QA_pairs is None:
            return None

        # Step 3: QA2D
        # to_predict = [qa['question'] + ' [SEP] ' + qa['answer'] for qa in QA_pairs] # original model
        to_predict = [qa['answer'] + '</s>' + qa['question'] for qa in QA_pairs]
        results = []
        # try:
        results = self.predict(to_predict)
        # except:
            # return None
        if len(results) == 0:
            print(f"zero length results for id: {id_}")
            return None

        assert len(results) == len(QA_pairs)

        claims_for_sample = OrderedDict()
        for ent, claim in zip(passage_entities, results):
            claims_for_sample[ent] = claim
        return claims_for_sample
    

    def generate_nei_claims(self, sample):
        pass


    def generate(self, claims_json, claim_type: str, save_every=0, cont=False):
        claim_type = claim_type.lower()
        assert claim_type in ["support", "refute", "nei"]
        start = 0
        if Path(claims_json).is_file():
            if cont:
                generated_claims = read_json(claims_json)
                print(f"file exists: {claims_json}, completed: {len(generated_claims)-1}/{len(self.corpus_recs)}")
                start = len(generated_claims)
            else:
                # print("--------------FIX!!!!!!!!!!!-------------------------")
                # generated_claims = read_json(claims_json)
                raise FileExistsError(f"File already exists: {claims_json} !!!")
        else:
            generated_claims = dict() # ordered since P3.7
        cnt = 1
        for sample in tqdm(self.corpus_recs[start:], initial=start, total=len(self.corpus_recs)):
            id_ = str(sample['id'])
            if claim_type == "support":
                claims = self.generate_supported_claims(sample)
            elif claim_type == "refute":
                claims = self.generate_refute_local_claims(sample)
            elif claim_type == "nei":
                claims = self.generate_nei_claims(sample)
            if claims is None:
                claims = {}
            generated_claims[id_] = claims
            cnt += 1
            if save_every > 0 and cnt % save_every == 0:
                write_json(claims_json, generated_claims, mkdir=True)

        write_json(claims_json, generated_claims, mkdir=True)